# Singularity Videos - Testing

### Pablo Jimeno - 2018

Using data from the Wide-field Infrared Survey Explorer (WISE) [https://wise.ssl.berkeley.edu]

In [ ]:
test_run = True

## Load libraries:

Select the working directory where folders will be created.

In [ ]:
## Set the project directory.
#project_dir = '/path/to/project/dir'
project_dir = '/home/pablo/VideoArte/singularity_video'


## Loads code functions and creates folders.
%run './utils/utils_main.ipynb'
create_folders(new_project=True) # Set "new_project=True" if you want to start off clean.

## Loads default configuration. To edit more technical defaults, check "utils_main" notebook.
config = Configuration()

## Configure video:

This is the configuration used for "Singularity #4".

In [ ]:
#-------------------------------------------------------------------------
# Aspect ratio: options are '16:9', '4:3', and '1:1'.
config.set_video_format('16:9')

#-------------------------------------------------------------------------
# Width of the video, in pixels [Default is 1280].
# In 16:9 format, 1280 implies frames of 1280x720, and 1920 implies 1920x1080
config.set_video_pix(1920)

#-------------------------------------------------------------------------
# Width of the horizontal black stripes in the top and the bottom of the video [Default is 60].
config.set_video_pixblackstripes(60)

#-------------------------------------------------------------------------
# Video properties: fps, fpi ("cluster interpolation speed"), and duration.
fps = 48 # Frames per second
fpi = 18 # Frames per cluster image
video_duration = 179.5 # Duration of video
config.set_video_info(fps, fpi, video_duration)

#-------------------------------------------------------------------------
# Especify dpi of screen [Default is 100]
#config.set_my_dpi(mydpi)

#-------------------------------------------------------------------------
# Beginning & end properties, and credits duration.
config.set_intro(1.5, 3.5, 3.) # Intro credits duration: (no text, text, no text) in seconds.
config.set_credits(3., 8., 2.) # Final credits duration: (no text, text, no text) in seconds.
config.set_t_offset_start(0.05) # Offset at start. This needs to be hand-tuned to sincronize audio-video.
config.set_t_black_start(0.05) # Black frame offset at start. This needs be hand-tuned to sincronize audio-video.

#-------------------------------------------------------------------------
# Display info of video:
config.display_video_info()

## Load redMaPPer cluster catalog:

Catalogue should be placed in the "cluster_catalogue" project folder.

In [ ]:
#redm_filedir = '/path/to/catalogue/file.fits' 
redm_filedir = '%s/cluster_catalogue/redMaPPer_DR12_v6.3.fits' % project_dir # Change if using another catalogue.
raw_redm = create_raw_sample()
redm = create_redm_subsample(raw_redm, config.rich_cutoff, config.z_min, config.z_max)

## Load audio file & process it:

In [ ]:
audio_file = '%s/sound_samples/singularity_soundtrack.wav' % project_dir

raw_audio_data = wavfile.read(audio_file)
audio_rate, audio_n_samples, audio_data = wav_analysis(raw_audio_data)
audio_data = audio_data[:,0] # Work with channel 1 from now on

config.set_audio_info(audio_rate, audio_n_samples)

## Configure the effects on the video as a function of the audio clip features:

This part needs to be hand-tuned for each different audio clip and desired effects. Seems obscure, but it's not. Just play with these numbers and display the result.

This is the configuration used for "Singularity #4".

In [ ]:
#-------------------------------------------------------------------------
# Drum beat
drum = Sonido(config, 'drum')
drum.set_start(12.05)
drum.set_rate(0.75)
drum.set_nbeats(16 + 32*4)

for i in range(16):
    drum.add_effect_gauss_bump(1., 0.09, drum.start + i*drum.rate)
    
for i in range(32):
    drum.add_effect_gauss_bump(1., 0.09, drum.start + 17*drum.rate + i*4*drum.rate + 0*drum.rate)
    drum.add_effect_gauss_bump(1., 0.09, drum.start + 17*drum.rate + i*4*drum.rate + 1*drum.rate)
    drum.add_effect_gauss_bump(1., 0.09, drum.start + 17*drum.rate + i*4*drum.rate + 2*drum.rate)
    
    
#-------------------------------------------------------------------------
# Sierra
sierra = Sonido(config, 'sierra')
sierra.set_start(24.15) 
sierra.set_rate(3.)
sierra.set_nbeats(41)

for i in range(sierra.nbeats):
    sierra.add_effect_step(1., 0.65, sierra.start + i*sierra.rate)
    

#-------------------------------------------------------------------------
# Platos - BH size
platos = Sonido(config, 'platos')
platos.set_start(48.)
platos.set_end(156.3)
platos.set_rate(0.75/8)
platos.set_nbeats(96)

platos.add_constant(0.3, platos.start, platos.end)

for i in range(4*platos.nbeats):
    platos.add_effect_prop_gauss_bump(0.1, 0.025, platos.start + i*platos.rate*2.)
for i in range(platos.nbeats):
    platos.add_effect_prop_gauss_bump(0.1, 0.025, platos.start + platos.rate + i*platos.rate*8.)

    
#-------------------------------------------------------------------------
# White hole
bateria = Sonido(config, 'bateria')
bateria.add_effect_step(1, 0.4, 156+0.75/2)


#-------------------------------------------------------------------------
# Bateria 72 -
platos.add_slope(0., 0.6, 72., 96.)
platos.add_constant(0.6, 96., 156.3+0.75/2)


#-------------------------------------------------------------------------
# Horns (Platos)
for i in range(8):
    platos.add_slope(i*1./8, (i+3)*1./8, 96.45 + i*3., 96.45 + i*3. + 1.5)
    platos.add_slope((i+3)*1./8, (i+1)*1./8, 96.45 + i*3. + 1.5, 96.45 + (i+1)*3.)
    
platos.add_slope(1.1, 3.4, 120., 144.)
platos.add_slope(3.4, 25., 144.45, 156.3+0.75/2)


#-------------------------------------------------------------------------
# Final
final = Sonido(config, 'final')
final.set_start(156.05+0.75/2) 
final.set_rate(0.75)
final.set_nbeats(np.int(24/0.75))

final.add_constant(-20., 0., final.start)
final.add_constant(0.1, final.start, config.video_duration)

for i in range(final.nbeats):
    final.add_effect_gauss_bump(20., 0.1, final.start + i*final.rate)
    
    
#-------------------------------------------------------------------------
# Fade to black
ftb = Sonido(config, 'ftb')
ftb.set_start(config.video_duration - 10.) 
ftb.set_end(config.video_duration)

ftb.add_constant(1., 0., ftb.start)
ftb.add_constant(-1., 156.2+0.75/2, 156.4+0.75/2)

ftb.add_slope(1., 0., ftb.start, ftb.end)


#-------------------------------------------------------------------------
# Clear overlaps:    

#drum.add_amp(-1000.*sierra.amp_frames)
platos.add_amp(-1000.*sierra.amp_frames)
final.add_amp(-1000.*bateria.amp_frames)
bateria.add_amp(-1000.*sierra.amp_frames)


#=================================================================================================================
## Use this to plot and check the sound/effects sequence.
if 0:
    sounds = [drum, sierra, platos, bateria, final, ftb] # The sound/effects you want to check
    v_start, v_end = 115., 159 # x-axis range
    max_val = 20. # y-axis max value
    plot_sounds_amplitude(config, max_val, audio_data, sounds, v_start, v_end)

## Download images from WISE server and process them:

(This may take some time if it is the first time it runs)

In [ ]:
if 1:
    generate_new_images(delete_fits=False, delete_npys=False, delete_pngs=False)
    wise_band = 1 # WISE satellite infrared bands: 1, 2, 3 or 4 (Recommended: 1 or 2).
    clu_images = 1000 # Pick a number large enough to have some sampling variance in each run.
    wise_meta = generate_images(redm, config, wise_band, clu_images=clu_images, produce_png=False)    

## Create interpolation:

Create new random realization of sequence of images, taking into account two sounds, and create the interpolated data-frames used to generate the final effect-frames.

(This may take some time).

In [ ]:
if 1:
    sounds = [drum, sierra]
    sequence_ids = order_images(wise_meta, config, sounds)
    create_interp_frames(sequence_ids, config, wise_band, renew=False)

## Test effects on particular frames:

Play with the numbers to adjust effects.

In [ ]:
frame_check = 150 # Choose frame number

frame = Frame(config, frame_check)
frame.load()
image = frame.image_data

#=================================================================================================================
## Choose effect to display:
#image = add_gaussian_2D(image, width, amp)
#image = add_whitehole(image, origin_image, width)
#image = add_blackmask(image, width, amp)
#image = add_fade_ring(image, width1, width2, amp1, amp2)
#image = add_blackhole(image, width, amp)
#image = convolve_image(image, size_kernel=5.)
#image = apply_aperture_mask(image, aperture=1., inverse=False)
#image = fft_filtering(image, filter_aperture, inverse)
#image = fancy_mask_effect(image)
#image = crop_borders_effect(image, crop_amp)
#image = norm_image_percentil(image, perc_down=0., perc_up=100., perc_norm_up=-1)
#image = norm_image_absolute(image, nabs_down=1., nabs_up=np.inf)
#image = norm_image_histogram(image, norm_down=1., norm_up=20.)
#image = effect_base(image)
#image = effect_mask(image, amp)
#image = effect_fft(image, amp, aperture)
#image = effect_fft_inv(image, amp, aperture)
#image = effect_blackhole(image, width)
#image = effect_whitehole(image, width)
#image = effect_blackwhitehole(image, width, whiteon)
#image = effect_singularity(image, amp, aperture)
#image = effect_singularity_v1(image, amp, aperture)
#image = effect_fade_to_black(image, amp)

image = effect_blackhole(image, 2.)

#-------------------------------------------------------------------------
## Prepare image:
image = display_image(config, image)
image = effect_prepare_border(image, 10.)

#-------------------------------------------------------------------------
## Set plotting options:
frame.set_image_data(image)
frame.set_img_v_max(10.)
frame.set_img_cmap('bone')

#=================================================================================================================
## Plot output:
frame.save_png(test=True)